# Imports

In [ ]:
import numpy as np
import torch
from tqdm import tqdm
import pickle
from data import *
from utils.analysis import *
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


## Absolute Measurement Analysis Between 2Bar-GrooveIQ and 2bar-GMD

### GrooveIQ Dataset

In [ ]:
dataset_root = "dataset/serialized"
giq_paths    = ['merged_ts=4-4_2bar_tr0.80-va0.10-te0.10_val.pkl', 'merged_ts=4-4_2bar_tr0.80-va0.10-te0.10_test.pkl', 'merged_ts=4-4_2bar_tr0.80-va0.10-te0.10_train.pkl']
name         = "GrooveIQ"

dataset = []
for path in giq_paths:
    with open(os.path.join(dataset_root, path), "rb") as f:
        dataset.extend(pickle.load(f))

GrooveIQStats = DataStats()
GrooveIQStats.set_name(name = name)

for datapoint in tqdm(dataset):
    GrooveIQStats.accumulate_dict(datapoint)

#GrooveIQStats.visualize()
#GrooveIQStats.summarize()

#### Feature Statistics

In [ ]:
## Absolute Measurements
giq_eval_dict = defaultdict(list)

for sample in tqdm(GrooveIQStats.all_samples):
    descriptors = sample.descriptors.descriptors
    for key, value in descriptors.items():
        giq_eval_dict[key].append(value)

giq_eval_dict = {key: np.array(value) for key, value in giq_eval_dict.items()}

for key, value in giq_eval_dict.items():
    print(f"{key:30}: mean={np.mean(value):.2f}, std={np.std(value):.2f}")


### GMD Dataset

In [ ]:
GMD_paths    = ['2barGMD.pkl']
name         = "2barGMD"

dataset = []
for path in GMD_paths:
    with open(os.path.join(dataset_root, path), "rb") as f:
        dataset.extend(pickle.load(f))

GMDStats = DataStats()
GMDStats.set_name(name = name)

for datapoint in tqdm(dataset):
    GMDStats.accumulate_dict(datapoint)

#GMDStats.summarize()
#GMDStats.visualize()

#### Feature Statistics

In [ ]:
## Absolute Measurements
gmd_eval_dict = defaultdict(list)

for sample in tqdm(GMDStats.all_samples):
    descriptors = sample.descriptors.descriptors
    for key, value in descriptors.items():
        gmd_eval_dict[key].append(value)

for key, value in gmd_eval_dict.items():
    print(f"{key:30}: mean={np.mean(value):.2f}, std={np.std(value):.2f}")

## Note Loss Analysis

In [ ]:
dataset_root = "dataset/serialized"
giq_paths    = ['merged_ts=4-4_2bar_tr0.80-va0.10-te0.10_val.pkl', 'merged_ts=4-4_2bar_tr0.80-va0.10-te0.10_test.pkl', 'merged_ts=4-4_2bar_tr0.80-va0.10-te0.10_train.pkl']
name         = "GrooveIQ"

dataset = []
for path in giq_paths:
    with open(os.path.join(dataset_root, path), "rb") as f:
        dataset.extend(pickle.load(f))

GrooveIQStats = DataStats()
GrooveIQStats.set_name(name = name)

for datapoint in tqdm(dataset):
    GrooveIQStats.accumulate_dict(datapoint)

note_loss_per_style = {key: [] for key in GrooveIQStats.style_map.keys()}

for sample in tqdm(GrooveIQStats.all_samples):
    _, stats = sample.feature.to_fixed_grid(steps_per_quarter=4)
    note_loss_per_style[sample.style].append(stats['note_loss']/stats['total_notes'])

for style, losses in note_loss_per_style.items():
    print(f"{style:30}: mean={np.mean(losses):.3f}, std={np.std(losses):.3f}")

plot_violin_distribution(note_loss_per_style, group_label='Style', feature="Note Loss")

## Model Quantitative Analysis

### Load Eval Samples

In [ ]:
# Load samples from pickle
with open("dataset/serialized/eval_samples.pkl", "rb") as f:
    samples = pickle.load(f)

### Load Checkpoints


In [ ]:
model1_l1_causal    = load_checkpoint('expts/giq_exp1_l1_causal', 'checkpoint-ep4-model')
model2_l1_ncausal   = load_checkpoint('expts/giq_exp2_l1_noncausal', 'checkpoint-ep4-model')
model3_grp_causal   = load_checkpoint('expts/giq_exp3_group_causal', 'checkpoint-ep4-model')
model4_grp_ncausal  = load_checkpoint('expts/giq_exp4_group_noncausal', 'checkpoint-ep4-model')
model5_heur_causal  = load_checkpoint('expts/giq_exp5_heur_causal', 'checkpoint-ep4-model')
model6_heur_ncausal = load_checkpoint('expts/giq_exp6_heur_noncausal', 'checkpoint-ep4-model')

### Eval Alignment

In [ ]:
model1_alignment_map = eval_alignment(samples, model1_l1_causal, is_heuristic=True)
model2_alignment_map = eval_alignment(samples, model2_l1_ncausal, is_heuristic=True)
model3_alignment_map = eval_alignment(samples, model3_grp_causal, is_heuristic=True)
model4_alignment_map = eval_alignment(samples, model4_grp_ncausal, is_heuristic=True)
model5_alignment_map = eval_alignment(samples, model5_heur_causal, is_heuristic=True)
model6_alignment_map = eval_alignment(samples, model6_heur_ncausal, is_heuristic=True)


In [ ]:
print(f"Model 1 Alignment Map: {"".join(f'\n\t{key}: Mean: {np.mean(model1_alignment_map[key])}, Std: {np.std(model1_alignment_map[key])}' for key in remove_nan(model1_alignment_map).keys())}")
print(f"Model 2 Alignment Map: {"".join(f'\n\t{key}: Mean: {np.mean(model2_alignment_map[key])}, Std: {np.std(model2_alignment_map[key])}' for key in remove_nan(model2_alignment_map).keys())}")
print(f"Model 3 Alignment Map: {"".join(f'\n\t{key}: Mean: {np.mean(model3_alignment_map[key])}, Std: {np.std(model3_alignment_map[key])}' for key in remove_nan(model3_alignment_map).keys())}")
print(f"Model 4 Alignment Map: {"".join(f'\n\t{key}: Mean: {np.mean(model4_alignment_map[key])}, Std: {np.std(model4_alignment_map[key])}' for key in remove_nan(model4_alignment_map).keys())}")
print(f"Model 5 Alignment Map: {"".join(f'\n\t{key}: Mean: {np.mean(model5_alignment_map[key])}, Std: {np.std(model5_alignment_map[key])}' for key in remove_nan(model5_alignment_map).keys())}")
print(f"Model 6 Alignment Map: {"".join(f'\n\t{key}: Mean: {np.mean(model6_alignment_map[key])}, Std: {np.std(model6_alignment_map[key])}' for key in remove_nan(model6_alignment_map).keys())}")

In [ ]:
hit_corrs_map = {
    "Model 1" : model1_alignment_map['hit_corrs'],
    "Model 2" : model2_alignment_map['hit_corrs'],
    "Model 3" : model3_alignment_map['hit_corrs'],
    "Model 4" : model4_alignment_map['hit_corrs'],
    "Model 5" : model5_alignment_map['hit_corrs'],
    "Model 6" : model6_alignment_map['hit_corrs']
}

peak_lags_map = {
    "Model 1" : model1_alignment_map['peak_lags'],
    "Model 2" : model2_alignment_map['peak_lags'],
    "Model 3" : model3_alignment_map['peak_lags'],
    "Model 4" : model4_alignment_map['peak_lags'],
    "Model 5" : model5_alignment_map['peak_lags'],
    "Model 6" : model6_alignment_map['peak_lags']
}


In [ ]:
plot_violin_distribution(hit_corrs_map, group_label="Model", feature="Hit Density Correlation", title_fontsize=18, label_fontsize=16, tick_fontsize=18)

In [ ]:
plot_violin_distribution(peak_lags_map, group_label='Model', feature='Temporal Cross-Correlation Lag', title_fontsize=18, label_fontsize=16, tick_fontsize=18)

### Evaluate Features


In [ ]:
eval_samples_features = {
        'total_hits' : [],
        'total_density' : [],
        'total_complexity' : [],
        'total_average_intensity' : [],
        'lowness' : [],
        'midness' : [],
        'highness' : [],
        'combined_syncopation' : [],
        'polyphonic_syncopation' : [],
        'laidbackness' : [],
        'swingness' : [],
        'timing_accuracy' : [],
}

for sample in tqdm(samples):
    sample, grid, button_hvo = sample['sample'], sample['grid'], sample['button_hvo']
    gt_drum_feature = sample.feature
    gt_descriptors = FeatureDescriptors(gt_drum_feature)
    total_hits = torch.sum(grid[:, :, 0] > 0)
    eval_samples_features['total_hits'].append(int(total_hits.item()))
    for key in eval_samples_features.keys():
        if key == 'total_hits':
            continue
        eval_samples_features[key].append(gt_descriptors.descriptors[key])

print(f"Ground Truth: {' '.join([f'\n\t{key}: {np.mean(eval_samples_features[key])}' for key in eval_samples_features.keys()])}")

In [ ]:
model1_feature_map = eval_features(samples, model1_l1_causal, is_heuristic=True)
model2_feature_map = eval_features(samples, model2_l1_ncausal, is_heuristic=True)
model3_feature_map = eval_features(samples, model3_grp_causal, is_heuristic=True)
model4_feature_map = eval_features(samples, model4_grp_ncausal, is_heuristic=True)
model5_feature_map = eval_features(samples, model5_heur_causal, is_heuristic=True)
model6_feature_map = eval_features(samples, model6_heur_ncausal, is_heuristic=True)

In [ ]:
print(f"Model 1: {' '.join([f'\n\t{key}: {np.mean(model1_feature_map[key])}' for key in model1_feature_map.keys()])}")
print(f"Model 2: {' '.join([f'\n\t{key}: {np.mean(model2_feature_map[key])}' for key in model2_feature_map.keys()])}")
print(f"Model 3: {' '.join([f'\n\t{key}: {np.mean(model3_feature_map[key])}' for key in model3_feature_map.keys()])}")
print(f"Model 4: {' '.join([f'\n\t{key}: {np.mean(model4_feature_map[key])}' for key in model4_feature_map.keys()])}")
print(f"Model 5: {' '.join([f'\n\t{key}: {np.mean(model5_feature_map[key])}' for key in model5_feature_map.keys()])}")
print(f"Model 6: {' '.join([f'\n\t{key}: {np.mean(model6_feature_map[key])}' for key in model6_feature_map.keys()])}")

In [ ]:
systems = {
    'Model 1' : model1_feature_map,
    'Model 2' : model2_feature_map,
    'Model 3' : model3_feature_map,
    'Model 4' : model4_feature_map,
    'Model 5' : model5_feature_map,
    'Model 6' : model6_feature_map
}

baseline_dict = eval_samples_features

dists = compute_per_feature_distances(systems, baseline_dict)
plot_kld_oa_per_feature(dists, system_names=list(systems.keys()), feature_title_fontsize=20, axis_label_fontsize=20, tick_fontsize=10, legend_fontsize=20)

In [ ]:
total_hits_map = {
    "Ground Truth" : eval_samples_features['total_hits'],
    "Model 1" : model1_feature_map['total_hits'],
    "Model 2" : model2_feature_map['total_hits'],
    "Model 3" : model3_feature_map['total_hits'],
    "Model 4" : model4_feature_map['total_hits'],
    "Model 5" : model5_feature_map['total_hits'],
    "Model 6" : model6_feature_map['total_hits']
}

In [ ]:
plot_violin_distribution_wrt_gt(total_hits_map, feature="Total Hits", title_fontsize=18, label_fontsize=16, tick_fontsize=18)

### Evaluate Reconstruction

In [ ]:
model1_recons_map = eval_recons(samples, model1_l1_causal, is_heuristic=False)
model2_recons_map = eval_recons(samples, model2_l1_ncausal, is_heuristic=False)
model3_recons_map = eval_recons(samples, model3_grp_causal, is_heuristic=False)
model4_recons_map = eval_recons(samples, model4_grp_ncausal, is_heuristic=False)
model5_recons_map = eval_recons(samples, model5_heur_causal, is_heuristic=True)
model6_recons_map = eval_recons(samples, model6_heur_ncausal, is_heuristic=True)

In [ ]:
print(f"Model 1: hit_ppv: {np.mean(model1_recons_map['hit_ppv']) :.2f}, hit_tpr: {np.mean(model1_recons_map['hit_tpr']) :.2f}, hit_f1: {np.mean(model1_recons_map['hit_f1']) :.2f}, velocity_mae: {np.mean(model1_recons_map['velocity_mae']) :.2f}, offset_mae: {np.mean(model1_recons_map['offset_mae']) :.2f}")
print(f"Model 2: hit_ppv: {np.mean(model2_recons_map['hit_ppv']) :.2f}, hit_tpr: {np.mean(model2_recons_map['hit_tpr']) :.2f}, hit_f1: {np.mean(model2_recons_map['hit_f1']) :.2f}, velocity_mae: {np.mean(model2_recons_map['velocity_mae']) :.2f}, offset_mae: {np.mean(model2_recons_map['offset_mae']) :.2f}")
print(f"Model 3: hit_ppv: {np.mean(model3_recons_map['hit_ppv']) :.2f}, hit_tpr: {np.mean(model3_recons_map['hit_tpr']) :.2f}, hit_f1: {np.mean(model3_recons_map['hit_f1']) :.2f}, velocity_mae: {np.mean(model3_recons_map['velocity_mae']) :.2f}, offset_mae: {np.mean(model3_recons_map['offset_mae']) :.2f}")
print(f"Model 4: hit_ppv: {np.mean(model4_recons_map['hit_ppv']) :.2f}, hit_tpr: {np.mean(model4_recons_map['hit_tpr']) :.2f}, hit_f1: {np.mean(model4_recons_map['hit_f1']) :.2f}, velocity_mae: {np.mean(model4_recons_map['velocity_mae']) :.2f}, offset_mae: {np.mean(model4_recons_map['offset_mae']) :.2f}")
print(f"Model 5: hit_ppv: {np.mean(model5_recons_map['hit_ppv']) :.2f}, hit_tpr: {np.mean(model5_recons_map['hit_tpr']) :.2f}, hit_f1: {np.mean(model5_recons_map['hit_f1']) :.2f}, velocity_mae: {np.mean(model5_recons_map['velocity_mae']) :.2f}, offset_mae: {np.mean(model5_recons_map['offset_mae']) :.2f}")
print(f"Model 6: hit_ppv: {np.mean(model6_recons_map['hit_ppv']) :.2f}, hit_tpr: {np.mean(model6_recons_map['hit_tpr']) :.2f}, hit_f1: {np.mean(model6_recons_map['hit_f1']) :.2f}, velocity_mae: {np.mean(model6_recons_map['velocity_mae']) :.2f}, offset_mae: {np.mean(model6_recons_map['offset_mae']) :.2f}")

In [ ]:
hit_ppv_map = {
    "Model 1" : model1_recons_map['hit_ppv'],
    "Model 2" : model2_recons_map['hit_ppv'],
    "Model 3" : model3_recons_map['hit_ppv'],
    "Model 4" : model4_recons_map['hit_ppv'],
    "Model 5" : model5_recons_map['hit_ppv'],
    "Model 6" : model6_recons_map['hit_ppv']
}

plot_violin_distribution(hit_ppv_map, group_label='Model', feature="PPV Hits", title_fontsize=18, label_fontsize=16, tick_fontsize=18)

In [ ]:
hit_tpr_map = {
    "Model 1" : model1_recons_map['hit_tpr'],
    "Model 2" : model2_recons_map['hit_tpr'],
    "Model 3" : model3_recons_map['hit_tpr'],
    "Model 4" : model4_recons_map['hit_tpr'],
    "Model 5" : model5_recons_map['hit_tpr'],
    "Model 6" : model6_recons_map['hit_tpr']
}

plot_violin_distribution(hit_tpr_map, group_label='Model', feature="TPR Hits", title_fontsize=18, label_fontsize=16, tick_fontsize=18)

In [ ]:
velocity_mae_map = {
    "Model 1" : model1_recons_map['velocity_mae'],
    "Model 2" : model2_recons_map['velocity_mae'],
    "Model 3" : model3_recons_map['velocity_mae'],
    "Model 4" : model4_recons_map['velocity_mae'],
    "Model 5" : model5_recons_map['velocity_mae'],
    "Model 6" : model6_recons_map['velocity_mae']
}

plot_violin_distribution(velocity_mae_map, group_label='Model', feature="Velocity MAE", title_fontsize=18, label_fontsize=16, tick_fontsize=18)

In [ ]:
offset_mae_map = {
    "Model 1" : model1_recons_map['offset_mae'],
    "Model 2" : model2_recons_map['offset_mae'],
    "Model 3" : model3_recons_map['offset_mae'],
    "Model 4" : model4_recons_map['offset_mae'],
    "Model 5" : model5_recons_map['offset_mae'],
    "Model 6" : model6_recons_map['offset_mae']
}

plot_violin_distribution(offset_mae_map, group_label='Model', feature="Offset MAE", title_fontsize=18, label_fontsize=16, tick_fontsize=18)